In [1]:
# %% [markdown]
# # Trading Strategy Analysis System

# %% [markdown]
# ## 1. Import Required Libraries and Modules

# %%
# Standard library imports
from typing import Dict, List, Optional, Tuple
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Data analysis imports
import pandas as pd
import numpy as np
import yfinance as yf

# Visualization imports
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('seaborn')
import seaborn as sns 
sns.set()
# Local module imports
from strategies.bollinger_bounce import BollingerBounceStrategy
from strategies.moving_average_cross import MovingAverageCrossStrategy
from strategies.support_resistance import SupportResistanceStrategy

from utils.data_processor import DataProcessor
from utils.performance_metrics import PerformanceAnalyzer
from utils.risk_manager import RiskManager
from utils.visualizer import Visualizer
from utils.report_generator import ReportGenerator

# %% [markdown]
# ## 2. Configuration and Settings

# %%
# Trading parameters
INITIAL_CAPITAL = 100000
START_DATE = "2023-01-01"
END_DATE = "2024-01-03"
SYMBOLS = ["EURUSD=X", "GBPUSD=X", "USDJPY=X"]

# Strategy parameters
STRATEGY_CONFIGS = {
    'bollinger_bounce': {
        'bb_period': 20,
        'bb_std': 2.0,
        'squeeze_factor': 0.8,
        'rsi_period': 14
    },
    'moving_average_cross': {
        'fast_period': 10,
        'slow_period': 30,
        'exit_bars': 5
    },
    'support_resistance': {
        'window': 20,
        'threshold': 0.02,
        'touch_count': 3
    }
}

# Risk parameters
RISK_PARAMS = {
    'max_position_size': 0.02,
    'max_portfolio_risk': 0.02,
    'max_correlation': 0.7,
    'max_drawdown': 0.20
}
# %% [markdown]
# ## 3. System Components Initialization

# %%
# %% [markdown]
# ## 3. System Components Initialization

# %%
class TradingSystem:
    """Main trading system interface"""
    
    def __init__(self, initial_capital: float = 100000):
        """Initialize trading system components"""
        self.initial_capital = initial_capital
        
        # Initialize components
        self.data_processor = DataProcessor()
        self.risk_manager = RiskManager(**RISK_PARAMS)
        self.performance_analyzer = PerformanceAnalyzer()
        self.visualizer = Visualizer()
        self.report_generator = ReportGenerator()
        
        # Initialize strategies with configurations
        self.strategies = {
            'bollinger_bounce': BollingerBounceStrategy(**STRATEGY_CONFIGS['bollinger_bounce']),
            'moving_average_cross': MovingAverageCrossStrategy(**STRATEGY_CONFIGS['moving_average_cross']),
            'support_resistance': SupportResistanceStrategy(**STRATEGY_CONFIGS['support_resistance'])
        }
        
        self.results = {}
    
    def run_backtest(self, 
                    symbol: str,
                    strategy_name: str,
                    start_date: str,
                    end_date: str) -> Dict[str, any]:
        """
        Run backtest for a single symbol and strategy
        
        Parameters:
        -----------
        symbol : str
            Trading symbol (e.g., 'EURUSD=X')
        strategy_name : str
            Name of strategy to use
        start_date : str
            Start date for backtest
        end_date : str
            End date for backtest
            
        Returns:
        --------
        Dict[str, any]
            Dictionary containing backtest results
        """
        print(f"\nRunning backtest for {symbol} using {strategy_name} strategy...")
        
        # Load and process data
        data = self.data_processor.load_data(symbol, start_date, end_date)
        
        # Generate signals
        strategy = self.strategies[strategy_name]
        data = strategy.generate_signals(data)
        
        # Apply risk management
        data = self.risk_manager.apply_position_sizing(data, self.initial_capital)
        
        # Calculate performance metrics
        metrics = self.performance_analyzer.calculate_metrics(data)
        
        # Generate visualizations
        figures = self.visualizer.create_strategy_dashboard(data, metrics)
        
        # Generate report
        report = self.report_generator.generate_report(
            df=data,
            metrics=metrics,
            figures=figures,
            strategy_name=strategy_name,
            symbol=symbol
        )
        
        return {
            'data': data,
            'metrics': metrics,
            'figures': figures,
            'report': report
        }
    
    def optimize_strategy(self,
                         symbol: str,
                         strategy_name: str,
                         param_ranges: Dict[str, any]) -> Dict[str, any]:
        """
        Optimize strategy parameters
        
        Parameters:
        -----------
        symbol : str
            Trading symbol
        strategy_name : str
            Strategy to optimize
        param_ranges : Dict[str, any]
            Parameter ranges for optimization
            
        Returns:
        --------
        Dict[str, any]
            Optimization results
        """
        data = self.data_processor.load_data(symbol, START_DATE, END_DATE)
        strategy = self.strategies[strategy_name]
        
        return strategy.optimize_parameters(data, param_ranges)

# Initialize trading system
system = TradingSystem(INITIAL_CAPITAL)

# %% [markdown]
# ## 4. Run Backtests

# %%
# Run backtests for each symbol and strategy
results = {}

for symbol in SYMBOLS:
    results[symbol] = {}
    for strategy_name in system.strategies.keys():
        results[symbol][strategy_name] = system.run_backtest(
            symbol=symbol,
            strategy_name=strategy_name,
            start_date=START_DATE,
            end_date=END_DATE
        )

# %% [markdown]
# ## 5. Performance Analysis

# %%
def analyze_results(results: Dict) -> pd.DataFrame:
    """Analyze and compare strategy performance"""
    performance_data = []
    
    for symbol in results:
        for strategy in results[symbol]:
            metrics = results[symbol][strategy]['metrics']
            
            performance_data.append({
                'Symbol': symbol,
                'Strategy': strategy,
                'Total Return': metrics['Total_Return'],
                'Sharpe Ratio': metrics['Sharpe_Ratio'],
                'Max Drawdown': metrics['Max_Drawdown'],
                'Win Rate': metrics['Win_Rate']
            })
    
    return pd.DataFrame(performance_data)

# Generate performance summary
performance_summary = analyze_results(results)
print("\nPerformance Summary:")
print(performance_summary)

# %% [markdown]
# ## 6. Visualization

# %%
# Plot comparative performance
plt.figure(figsize=(15, 10))

for symbol in results:
    for strategy in results[symbol]:
        data = results[symbol][strategy]['data']
        returns = (1 + data['Strategy_Returns']).cumprod()
        plt.plot(returns.index, returns, 
                label=f'{symbol} - {strategy}')

plt.title('Comparative Strategy Performance')
plt.xlabel('Date')
plt.ylabel('Cumulative Returns')
plt.legend()
plt.grid(True)
plt.show()

# %% [markdown]
# ## 7. Strategy Optimization Example

# %%
# Optimize Bollinger Bounce strategy for EURUSD
param_ranges = {
    'bb_period': range(10, 31, 5),
    'bb_std': [1.5, 2.0, 2.5],
    'squeeze_factor': [0.6, 0.7, 0.8, 0.9]
}

optimization_results = system.optimize_strategy(
    symbol="EURUSD=X",
    strategy_name="bollinger_bounce",
    param_ranges=param_ranges
)

print("\nOptimization Results:")
print(f"Best Parameters: {optimization_results['best_parameters']}")
print(f"Best Sharpe Ratio: {optimization_results['best_sharpe']:.2f}")

# %% [markdown]
# ## 8. Generate Reports

# %%
# Generate and save reports for each backtest
for symbol in results:
    for strategy in results[symbol]:
        result = results[symbol][strategy]
        
        # Save HTML report
        filename = f"report_{symbol}_{strategy}.html"
        system.report_generator.save_report(
            result['report'],
            filename
        )
        
        # Generate PDF report
        pdf_filename = f"report_{symbol}_{strategy}.pdf"
        system.report_generator.generate_pdf_report(
            result['data'],
            result['metrics'],
            pdf_filename
        )

print("\nReports generated successfully!")

# %% [markdown]
# ## 9. Risk Analysis

# %%
def analyze_risk_metrics(results: Dict) -> pd.DataFrame:
    """Analyze risk metrics across strategies"""
    risk_data = []
    
    for symbol in results:
        for strategy in results[symbol]:
            data = results[symbol][strategy]['data']
            risk_metrics = system.risk_manager.calculate_risk_metrics(data)
            
            risk_data.append({
                'Symbol': symbol,
                'Strategy': strategy,
                'Current Risk Exposure': risk_metrics['Current_Risk_Exposure'],
                'Max Position Size': risk_metrics['Max_Position_Size'],
                'Volatility': risk_metrics['Volatility'],
                'VaR (95%)': risk_metrics['VaR_95']
            })
    
    return pd.DataFrame(risk_data)

# Generate risk analysis
risk_analysis = analyze_risk_metrics(results)
print("\nRisk Analysis:")
print(risk_analysis)

# %% [markdown]
# ## 10. Save Results

# %%
# Save performance and risk analysis to Excel
with pd.ExcelWriter('backtest_results.xlsx') as writer:
    performance_summary.to_excel(writer, sheet_name='Performance', index=False)
    risk_analysis.to_excel(writer, sheet_name='Risk', index=False)

print("\nResults saved to backtest_results.xlsx")

OSError: 'seaborn' is not a valid package style, path of style file, URL of style file, or library style name (library styles are listed in `style.available`)